## Object-Oriented Design: War Card Game

Now let's design a card game. If you've never played War before (or if you have), 
here's the variant of the game that we will play. The
deck is split evenly between two players. On each turn, both players
reveal their first card. The player showing the card with the highest rank takes both
cards and adds them to his or her deck, and reshuffles their hand. If there is a tie, the players
reveal their next cards. If there is no longer a tie, the player with
the highest rank takes all four cards on the revealed stack into their hand (and reshuffles their hand). 
Otherwise, both players continue revealing their next cards until the tie is broken. 
The game continues until one player has collected all 52 cards.

What classes and methods do we need? Each card has a rank and a suit,
so it makes sense to make a **Card** class with these attributes. In our
game, "greater" cards are those with higher rank; cards with the same
rank are "equal", no matter what suit they come from.

A **Hand** is a collection of Cards, so we'll make that another class. We can give
and take cards from a Hand, and also shuffle the cards in a Hand. We'll
want to know how many Cards are in a Hand, so we'll need a `num_cards` method,
among others.

A **Deck** is a special kind of Hand, populated with (in our case) the 
standard 52 cards. We can `deal` cards from a Deck.

A **Player** is also a Hand of cards, but with a (person) name also.

Finally, the **Game** class implements all of the game logic. This class
implements methods for dealing hands, taking a turn, and playing a
game to determine a winner.

### Card class

In [206]:
class Card():
    def __init__(self, rank, suit):
        """ rank: integer from 2 to (including) 14
            suit: 'S' for Spades, or 'H' for Hearts, 
                  or 'C' for Clubs, or 'D' for Diamonds
        """
        assert suit in {'S', 'H', 'C', 'D'}
        assert rank in range(2,15)
        self.rank = rank
        self.suit = suit
        
    suit_str = {'S': "\u2660", 'H': "\u2661", 'C': "\u2663", 'D': "\u2662"}
    rank_str = {n: str(n) for n in range(2,11)}
    rank_str[11] = 'J'; rank_str[12] = 'Q'; rank_str[13] = 'K'; rank_str[14] = 'A' 
    
    def __str__(self): # '2♡'
        return f"{self.rank_str[self.rank]}{self.suit_str[self.suit]}"
    
    def __repr__(self): # Card(2,'H')
        return f"Card({self.rank},'{self.suit}')"
    
    def __gt__(self, other):
        return self.rank > other.rank
    
    def __lt__(self, other):
        return self.rank < other.rank

    def __eq__(self, other):
        return self.rank == other.rank

Creating Cards, and their repr and str representations:

In [207]:
Card(2,'H') #new instance of Card

Card(2,'H')

In [208]:
repr(Card(2,'H'))

"Card(2,'H')"

In [209]:
str(Card(2,'H'))

'2♡'

Card comparisons -- based on rank, not suit:

In [210]:
Card(2,'H') == Card(2,'S') #want True

True

In [211]:
Card(2,'H') > Card(7,'S') #want False

False

In [212]:
Card(2,'H') < Card(7,'S') #want True

True

Some customized notions of equality:

In [213]:
Card(2,'H') is Card(2,'H') #want False; different instances

False

In [214]:
Card(2,'H') == Card(2,'H') #want True; based on rank

True

In [215]:
Card(2,'H') == Card(2,'S') #want True; based on rank

True

By implementing <, >, ==, we gain a powerful ability to use many python built-in functions, like `max`, `min`, `sorted`, etc.!

In [216]:
cards = [Card(3,'S'), Card(14,'D'), Card(10,'D'), Card(14,'H')]
print("cards:", cards) # note: str(<list>) creates a string of the list of <repr of list elements>
print("max:", max(cards))
print("min:", min(cards))
print("position of max card:", cards.index(max(cards)))
print("regular sorted:", [str(c) for c in sorted(cards)])
print("reverse sorted:", [str(c) for c in sorted(cards, reverse=True)])

cards: [Card(3,'S'), Card(14,'D'), Card(10,'D'), Card(14,'H')]
max: A♢
min: 3♠
position of max card: 1
regular sorted: ['3♠', '10♢', 'A♢', 'A♡']
reverse sorted: ['A♢', 'A♡', '10♢', '3♠']


### Hand class

In [217]:
import random
class Hand():
    def __init__(self, cards=[]):
        self.cards = [] #avoid aliasing; receive_cards into new empty list
        self.receive_cards(cards)

    def receive_cards(self, cards):
        """ Receive cards into hand """
        for c in cards:
            self.receive_card(c)
        
    def receive_card(self, card):
        """ Receive single card into bottom of hand """
        self.cards.append(card)

    def shuffle(self):
        """ Shuffle the deck by rearranging the cards in random order. """
        if self.cards:
            random.shuffle(self.cards)

    def give_card(self):
        """ Remove and return the card at the top of the hand. """
        assert self.num_cards > 0
        return self.cards.pop(0)
    
    def give_cards(self):
        """ Remove and yield cards from the top of the hand, until
            there are no more cards """
        while self.num_cards > 0:
            yield self.cards.pop(0)
        
    #make num_cards method look like an attribute
    @property
    def num_cards(self):
        return len(self.cards)

    def __iter__(self):
        """ Iterator over the cards in the hand. Does not remove
            the cards from the hand. """
        yield from self.cards

    def __repr__(self): # Hand([Card(2,'S')])
        return f"Hand({self.cards})"
    
    def __str__(self): # ['2♠']
        return str([str(card) for card in self.cards])

In [218]:
h = Hand()
h.receive_card(Card(2,'S'))
h.receive_cards([Card(13,'H'), Card(7,'C')])
print("hand:", h)
print("num_cards:", h.num_cards) #instead of h.num_cards()
print("give card:", h.give_card())
print("rest of cards:", [str(c) for c in h.give_cards()])
h

hand: ['2♠', 'K♡', '7♣']
num_cards: 3
give card: 2♠
rest of cards: ['K♡', '7♣']


Hand([])

### Deck class

In [219]:
class Deck(Hand):
    def __init__(self):
        super().__init__(Deck.build_deck())
        self.size = self.num_cards
        self.shuffle()

    @staticmethod
    def build_deck():
        """
        Return a list of 52 cards, as in a standard deck.

        Suits are "H" (Hearts), "S" (Spades), "C" (Clubs), "D" (Diamonds).
        Ranks in order of increasing strength the numbered cards 
        2-10, 11 Jack, 12 Queen, 13 King, and 14 Ace.
        """
        suits = {"H", "S", "C", "D"}
        return [Card(rank, suit) for rank in range(2,15) for suit in suits]

In [220]:
d = Deck()
print(d)
print("size of deck:", d.size)
print("give card:", d.give_card())
print("rest of cards:", [str(c) for c in d.give_cards()])

['10♠', 'J♠', 'A♣', '2♠', '2♣', '9♡', 'J♣', 'K♢', '8♢', 'Q♡', '3♢', '5♢', 'K♣', '5♠', '4♣', '10♢', '5♣', '7♡', 'Q♣', 'A♠', '8♣', '2♡', 'K♡', 'Q♢', '6♠', '3♡', '3♣', '6♢', '9♢', '6♡', '8♠', '7♢', '4♢', '7♠', 'J♢', 'K♠', 'Q♠', '10♣', 'A♢', '9♣', '10♡', '7♣', '9♠', '6♣', '5♡', 'J♡', '4♠', '2♢', '3♠', 'A♡', '8♡', '4♡']
size of deck: 52
give card: 10♠
rest of cards: ['J♠', 'A♣', '2♠', '2♣', '9♡', 'J♣', 'K♢', '8♢', 'Q♡', '3♢', '5♢', 'K♣', '5♠', '4♣', '10♢', '5♣', '7♡', 'Q♣', 'A♠', '8♣', '2♡', 'K♡', 'Q♢', '6♠', '3♡', '3♣', '6♢', '9♢', '6♡', '8♠', '7♢', '4♢', '7♠', 'J♢', 'K♠', 'Q♠', '10♣', 'A♢', '9♣', '10♡', '7♣', '9♠', '6♣', '5♡', 'J♡', '4♠', '2♢', '3♠', 'A♡', '8♡', '4♡']


### Player class

In [221]:
class Player(Hand):
    def __init__(self, name, cards=[]):
        super().__init__(cards)
        self.name = name
    
    def __repr__(self): # Player('Pam', [Card(10,'H')])
        return f"Player({repr(self.name)}, {[c for c in self]})"
    
    def __str__(self):  # <'Pam', ['10♡']>
        return f"<{repr(self.name)} has {[str(c) for c in self]}>"

In [222]:
p = Player('Pam')
p.receive_card(Card(10,'H'))
print(p)
p

<'Pam' has ['10♡']>


Player('Pam', [Card(10,'H')])

### Game class

In [223]:
class Game():
    def __init__(self, players):
        self.players = players
        self.deck = Deck()

    def deal(self):
        """ 
        Deal cards to both players. Each player takes one card at 
        a time from the deck.
        """
        while self.deck.num_cards > 0:
            for p in self.players:
                p.receive_card(self.deck.give_card())

    def turn(self, do_print=False):
        """
        Get cards from both players. The player with the higher 
        rank takes all the cards in the pile.
        """
        assert all(p.num_cards > 0 for p in self.players)
        pile = [p.give_card() for p in self.players]
   
        # If there is a tie, get the next cards and add them to the
        # top (front of) the cards pile, if any players still have cards
        while pile[0] == pile[1] and all(p.num_cards > 0 for p in self.players):
            if do_print: print("war:", [str(c) for c in pile], "TIE!")
            for i, p in enumerate(self.players):
                pile.insert(i, p.give_card())
        
        if pile[0] == pile[1]:
            # There is a tie, but at least one player ran out of cards,
            # and that player loses.
            winner = self.players[0] if self.players[0].num_cards > 0 else self.players[1]
        else:
            winner = self.players[0] if pile[0] > pile[1] else self.players[1]
        winner.receive_cards(pile)
        winner.shuffle()
        if do_print: print("war:", [str(c) for c in pile], "=>", winner.name)
        return winner

    def play(self, do_print=False):
        """
        Keep taking turns until a player has won (has all 52 cards).
        Return the winning player.
        """
        self.deal()
        if do_print: 
            for p in self.players: 
                print("at start", p)
                
        while True:
            winner = self.turn(do_print)
            if winner.num_cards == self.deck.size:
                break
    
        if do_print: print(winner.name + " wins!")
        return winner
        
    def play_n_times(self, n):
        for p in self.players:
            p.wins = 0 #interesting! Add instance attribute on the fly

        for i in range(n):
            winner = self.play()
            winner.wins += 1

            # Return all cards players are holding back to the deck
            for p in self.players:
                self.deck.receive_cards(p.give_cards())
            self.deck.shuffle()
            self.deal()
            
        print("\nPlayed", n, "hands")
        for p in self.players:
            print("  ", p.name, "wins:", p.wins)

### Let's try it out...

In [224]:
game = Game([Player("Amy"), Player("Brad")])
game.play(do_print = True)

at start <'Amy' has ['A♡', '6♢', 'Q♡', '8♢', '2♡', 'A♠', '3♢', '9♡', 'K♢', '10♣', '9♠', '5♡', '10♢', '2♠', '8♡', '7♡', '9♣', 'Q♣', 'Q♢', '9♢', '3♠', '3♣', 'J♡', '7♠', 'K♡', '10♠']>
at start <'Brad' has ['2♢', 'A♢', '5♢', 'J♠', '3♡', 'J♣', '7♢', '2♣', '8♠', '4♢', 'K♣', '6♣', '6♠', '6♡', '7♣', '4♡', '4♣', 'K♠', '5♠', '10♡', '4♠', 'A♣', 'Q♠', '8♣', '5♣', 'J♢']>
war: ['A♡', '2♢'] => Amy
war: ['A♡', 'A♢'] TIE!
war: ['7♡', '5♢', 'A♡', 'A♢'] => Amy
war: ['2♠', 'J♠'] => Brad
war: ['K♡', '8♠'] => Amy
war: ['3♢', 'J♣'] => Brad
war: ['9♢', '3♢'] => Amy
war: ['Q♢', '8♣'] => Amy
war: ['7♠', 'K♠'] => Brad
war: ['A♠', '6♠'] => Amy
war: ['2♢', '6♡'] => Brad
war: ['10♠', '7♢'] => Amy
war: ['J♡', 'J♢'] TIE!
war: ['7♢', 'J♣', 'J♡', 'J♢'] => Brad
war: ['9♡', '3♡'] => Amy
war: ['6♢', 'J♠'] => Brad
war: ['A♢', 'J♡'] => Amy
war: ['A♡', '4♣'] => Amy
war: ['8♢', '2♣'] => Amy
war: ['Q♢', '6♢'] => Amy
war: ['Q♢', '5♣'] => Amy
war: ['3♠', '4♢'] => Brad
war: ['2♡', '4♠'] => Brad
war: ['4♣', 'A♣'] => Brad
war: ['8♢

war: ['Q♢', '3♡'] => Amy
war: ['K♢', 'Q♣'] => Amy
war: ['K♢', '3♣'] => Amy
war: ['6♠', '9♢'] => Brad
war: ['9♠', 'K♡'] => Brad
war: ['8♠', '10♠'] => Brad
war: ['2♡', '7♢'] => Brad
war: ['8♢', 'A♣'] => Brad
war: ['7♡', 'K♠'] => Brad
war: ['Q♢', 'J♢'] => Amy
war: ['3♡', '7♡'] => Brad
war: ['5♢', '8♣'] => Brad
war: ['6♣', '6♠'] TIE!
war: ['K♢', 'K♡', '6♣', '6♠'] TIE!
war: ['10♣', '8♣', 'K♢', 'K♡', '6♣', '6♠'] => Amy
war: ['4♢', '4♣'] TIE!
war: ['A♢', '8♠', '4♢', '4♣'] => Amy
war: ['8♠', '7♡'] => Amy
war: ['A♡', '2♣'] => Amy
war: ['8♣', 'K♣'] => Brad
war: ['4♢', '8♢'] => Brad
war: ['A♡', '5♣'] => Amy
war: ['6♡', '9♢'] => Brad
war: ['J♡', '8♢'] => Amy
war: ['10♢', '5♠'] => Amy
war: ['J♠', 'J♣'] TIE!
war: ['7♠', 'K♠', 'J♠', 'J♣'] => Brad
war: ['2♣', '4♢'] => Brad
war: ['9♣', '5♢'] => Amy
war: ['5♠', '8♣'] => Brad
war: ['A♠', 'K♣'] => Amy
war: ['5♢', '7♢'] => Brad
war: ['6♢', 'K♠'] => Brad
war: ['9♣', '3♠'] => Amy
war: ['J♡', '6♡'] => Amy
war: ['7♡', '6♢'] => Amy
war: ['2♢', '5♠'] => Brad
war

Player('Amy', [Card(5,'D'), Card(13,'H'), Card(6,'D'), Card(6,'H'), Card(8,'S'), Card(9,'S'), Card(7,'D'), Card(4,'C'), Card(10,'H'), Card(2,'D'), Card(9,'H'), Card(2,'C'), Card(14,'S'), Card(3,'S'), Card(10,'C'), Card(5,'S'), Card(3,'D'), Card(11,'S'), Card(10,'D'), Card(9,'D'), Card(3,'C'), Card(3,'H'), Card(13,'S'), Card(7,'S'), Card(5,'H'), Card(12,'S'), Card(8,'C'), Card(6,'C'), Card(7,'H'), Card(14,'D'), Card(14,'H'), Card(14,'C'), Card(13,'D'), Card(11,'H'), Card(12,'H'), Card(7,'C'), Card(2,'S'), Card(12,'C'), Card(8,'D'), Card(4,'S'), Card(4,'H'), Card(8,'H'), Card(11,'C'), Card(4,'D'), Card(6,'S'), Card(5,'C'), Card(2,'H'), Card(10,'S'), Card(9,'C'), Card(13,'C'), Card(12,'D'), Card(11,'D')])

## Many games
Let's extend so we can run many games and see who wins the most.

In [225]:
game = Game([Player("Amy"), Player("Brad")])
game.play_n_times(3)
game.play_n_times(30)
game.play_n_times(100)


Played 3 hands
   Amy wins: 2
   Brad wins: 1

Played 30 hands
   Amy wins: 14
   Brad wins: 16

Played 100 hands
   Amy wins: 55
   Brad wins: 45


### Try with more than two players

In [ ]:
game = Game([Player("Amy"), Player("Brad"), Player("Carl")])
game.play_n_times(3)
game.play_n_times(30)
game.play_n_times(100)

Oops. We hard coded the game to only expect two players! We'll leave it as an exercise for the reader to go back and generalize `Game` to fix.